# Week 3 Assignment

## Code to parse the wikipage and build dataframe

In [1]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd

source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'lxml')
filename = 'wiki_scrape.csv'

# csv_file = open(filename, 'w')
# csv_writer = csv.writer(csv_file)
# headers = ['PostCode', 'Borough', 'Neighbourhood']

# get the table and its tbody from html 
wikitable = soup.find('table', class_='wikitable')
wikitbody = wikitable.find('tbody')
# loop through all rows in tbody
dataList=[]
for trow in wikitbody.find_all('tr'):
    # the tbody is returning first column as th - handle it by using as first row
    cols = trow.find_all('th')
    if (cols != []) : 
        headers=[cols[0].text.strip(), cols[1].text.strip(), cols[2].text.strip()]        
        # csv_writer.writerow(headers)
        
    else : 
        # now work on the columns 
        cols = trow.find_all('td')                 
        
        postcode = cols[0].text.strip()       
        
        # get borough value as it can be either a tag or just text
        if (cols[1].a != None) :  borough = cols[1].a.text.strip() 
        else :  borough = cols[1].text.strip()
       
        # get neighborhood and if it's value is 'not assigned', use borough value
        neighborhood = cols[2].text.strip() 
        if (neighborhood.lower() == 'not assigned') : neighborhood = borough
        
        # Skip loading 'not assigned' borough 
        if ( borough.lower() != "not assigned") :  
            # csv_writer.writerow([postcode, borough, neighborhood])
            dataList.append([postcode, borough, neighborhood])        
        
# csv_file.close()
# tor_df = pd.read_csv(filename)
tor_df = pd.DataFrame(dataList, columns=headers)
tor_df = tor_df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

#  Use the excel file to get the longitudes and lattitudes
lat_lng_df = pd.read_csv('http://cocl.us/Geospatial_data')
# Rename the first column for merging with Toronto dataframe
lat_lng_df = lat_lng_df.rename(index=str, columns={"Postal Code": "Postcode"})
tor_df_loc = pd.merge(tor_df, lat_lng_df, on=['Postcode'])
tor_df_loc.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [13]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

### Get the venues around Toronto

In [3]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
       # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [5]:
toronto_venues = getNearbyVenues(names=tor_df_loc['Borough'],
                                   latitudes=tor_df_loc['Latitude'],
                                   longitudes=tor_df_loc['Longitude']
                                  )

In [6]:
print(toronto_venues.shape)
# toronto_venues.head(10)
toronto_venues.groupby('Borough').count()

(2241, 7)


,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
Central Toronto,114,114,114,114,114,114
Downtown Toronto,1290,1290,1290,1290,1290,1290
East Toronto,121,121,121,121,121,121
East York,75,75,75,75,75,75
Etobicoke,72,72,72,72,72,72
Mississauga,11,11,11,11,11,11
North York,230,230,230,230,230,230
Queen's Park,39,39,39,39,39,39
Scarborough,85,85,85,85,85,85


### Analyze each borough in Toonto

In [7]:
# one hot encoding
toronto_one = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_one['Borough'] = toronto_venues['Borough'] 

# move neighborhood column to the first column
fixed_columns = [toronto_one.columns[-1]] + list(toronto_one.columns[:-1])
toronto_one = toronto_one[fixed_columns]

toronto_one.head()

,Borough,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
toronto_grouped = toronto_one.groupby('Borough').mean().reset_index()
toronto_grouped.head()

,Borough,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,...,0.008772,0.00000,0.000000,0.008772,0.000000,0.000000,0.0,0.000000,0.0,0.008772
1,Downtown Toronto,0.000775,0.000775,0.000775,0.000775,0.000775,0.000775,0.00155,0.00155,0.00155,...,0.012403,0.00155,0.000000,0.005426,0.000000,0.006202,0.0,0.000775,0.0,0.003101
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.024793
3,East York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,...,0.000000,0.00000,0.013333,0.000000,0.013333,0.000000,0.0,0.000000,0.0,0.013333
4,Etobicoke,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.013889,0.0,0.000000


In [9]:
import numpy as np
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

# Create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted


,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Park,Pizza Place,Café,Sushi Restaurant,Dessert Shop,Pub,Gym,Burger Joint
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Bar,Japanese Restaurant,Seafood Restaurant,American Restaurant
2,East Toronto,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Café,Brewery,Yoga Studio,Pizza Place,Bookstore,Pub
3,East York,Coffee Shop,Park,Pharmacy,Burger Joint,Sporting Goods Shop,Bank,Pizza Place,Fast Food Restaurant,Sandwich Place,Supermarket
4,Etobicoke,Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Fast Food Restaurant,Grocery Store,Gym,Park,Fried Chicken Joint,Beer Store
5,Mississauga,Hotel,Coffee Shop,Gym / Fitness Center,Middle Eastern Restaurant,Sandwich Place,Burrito Place,American Restaurant,Mediterranean Restaurant,Fried Chicken Joint,Drugstore
6,North York,Coffee Shop,Clothing Store,Fast Food Restaurant,Restaurant,Pizza Place,Japanese Restaurant,Grocery Store,Sandwich Place,Bank,Café
7,Queen's Park,Coffee Shop,Japanese Restaurant,Gym,Diner,Yoga Studio,Burrito Place,Italian Restaurant,Fast Food Restaurant,Smoothie Shop,Liquor Store
8,Scarborough,Fast Food Restaurant,Chinese Restaurant,Coffee Shop,Pizza Place,Breakfast Spot,Sandwich Place,Bakery,Thai Restaurant,Park,Playground
9,West Toronto,Bar,Café,Coffee Shop,Italian Restaurant,Bakery,Pizza Place,Restaurant,Breakfast Spot,Music Venue,Diner


### Cluster Boroughs

In [10]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = tor_df_loc

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Borough'), on='Borough')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,1,Fast Food Restaurant,Chinese Restaurant,Coffee Shop,Pizza Place,Breakfast Spot,Sandwich Place,Bakery,Thai Restaurant,Park,Playground
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,1,Fast Food Restaurant,Chinese Restaurant,Coffee Shop,Pizza Place,Breakfast Spot,Sandwich Place,Bakery,Thai Restaurant,Park,Playground
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1,Fast Food Restaurant,Chinese Restaurant,Coffee Shop,Pizza Place,Breakfast Spot,Sandwich Place,Bakery,Thai Restaurant,Park,Playground
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1,Fast Food Restaurant,Chinese Restaurant,Coffee Shop,Pizza Place,Breakfast Spot,Sandwich Place,Bakery,Thai Restaurant,Park,Playground
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,Fast Food Restaurant,Chinese Restaurant,Coffee Shop,Pizza Place,Breakfast Spot,Sandwich Place,Bakery,Thai Restaurant,Park,Playground


In [11]:
from geopy.geocoders import Nominatim

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronoto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronoto are 43.653963, -79.387207.


### # code for Toronto data map
<!--

'''
# create map of Toronot using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(tor_df_loc['Latitude'], tor_df_loc['Longitude'], tor_df_loc['Borough'], tor_df_loc['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto
'''
-->

In [12]:
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Borough'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

